# Fuentes

### Link: https://stats.stackexchange.com/questions/352036/what-should-i-do-when-my-neural-network-doesnt-learn
Sin palabras.

### Link: https://machinelearningmastery.com/how-to-configure-the-number-of-layers-and-nodes-in-a-neural-network/
Explicación sobre las redes neuronales con múltiples capas, y lo que más interesante me pareció fue la enumeración de diferentes enfoques para saber cómo proponer la cantidad de capas y las neurones por cada capa a utilizar en el modelo de la red neuronal.

### Link: https://medium.com/fintechexplained/what-are-hidden-layers-4f54f7328263
En este artículo hace una explicación de las capas de una red neuronal de múltiples capas, útil para diferencias capa de entrada, salida y capas ocultas o intermedias.

### Link: https://mmuratarat.github.io/2019-06-12/embeddings-with-numeric-variables-Keras
Este artículo es importante porque te explica cómo usar la API funcional de Keras, que fue útil a la hora de utilizar embeddings porque necesitabamos que las nuevas entradas del modelo generadas por la capa de embedding se juntaran con las demás variables del problema, para lo cual tuvimos que emplear una capa *concatenate*, que funciona al utilizar la API funciona de Keras.

### Link: https://stackoverflow.com/questions/61367382/plot-custom-data-with-tensorboard
Esta discusión de StackOverflow muestra un código de muestra de Python para registrar información personalizada usando TensorBoard, me resulto de gran utilidad para poder concentrar toda la información del proceso de entrenamiento, validación y evaluación de una red neuronal, cada vez que probaba, en un mismo lugar.

### Link: https://branyang.gitbooks.io/tfdocs/content/get_started/embedding_viz.html#metadata
Este link me sirvió para entender un poco más las opciones de visualización de embedding en TensorBoard, y para poder configurar el archivo de metadata para visualizar bien dónde quedaba cada uno de los valores de la variable o feature luego de la capa de embeddig.

### Link: https://machinelearningmastery.com/choose-an-activation-function-for-deep-learning/
Este artículo me sirvió para leer un poco más sobre funciones de activación, cómo se suelen usar en modelos con múltiples capas, algunos detalles me parecieron interesantes como el hecho de no cambiar la función activación a lo largo de las capas de un modelo (lo que usualmente se hace en la práctica según explican), y luego el hecho de que en las redes MLP suele predominar el uso de la **RELU**, aunque no entran en detalle con respecto a sus variantes pero sí dicen que se pueden utilizar. Esto me da la pauta de que debería mantener en las capas ocultas una **RELU** o alguna de sus variantes, pero no sigmoide o tangente hiperbólica porque pueden presentar un problema de **vanishing gradient**.

### Link: https://mlfromscratch.com/activation-functions-explained/#/
Otro artículo interesante que explica las funciones de activación, sus ventajas y desventajas, entre otras cuestiones. Es útil para tener más o menos una idea de qué problemas puede causar cada función de activación, me resultó útil para darme cuenta de algunas cosas que podían estar afectando al modelo.

# 1. Cargando base de datos

In [51]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

In [52]:
import matplotlib.pyplot as plt

In [53]:
import numpy as np

In [54]:
import importlib

In [55]:
import sys

In [56]:
sys.path.insert(0, '..')

In [57]:
sys.path.insert(0, '../..')

In [58]:
# Read the database from the .csv file into a pandas dataframe
df = pd.read_csv('../../databases/insurance.csv')

# 2. Preprocesamiento de los datos

In [59]:
from sklearn import preprocessing

## 2.1. Codificación de variables no numéricas o categóricas

In [60]:
# Create a label encoder for the sex variable or feature and create a new column in the dataframe 
# with the encoded version of the gender
sex_encoder = preprocessing.LabelEncoder()
sex_encoder.fit(df['sex'])
df['sex-encoded'] = sex_encoder.transform(df['sex'])

In [61]:
# Create a label encoder for the smoker variable or feature and create a new column in the dataframe
# with the encoded version of the smoker
smoker_encoder = preprocessing.LabelEncoder()
smoker_encoder.fit(df['smoker'])
df['smoker-encoded'] = smoker_encoder.transform(df['smoker'])

In [62]:
# Create a label encoder for the region variable or feature and create a new column in the dataframe
# with the encoded version of the region
region_encoder = preprocessing.LabelEncoder()
region_encoder.fit(df['region'])
df['region-encoded'] = region_encoder.transform(df['region'])

In [63]:
df.head()

,age,sex,bmi,children,smoker,region,charges,sex-encoded,smoker-encoded,region-encoded
0,19,female,27.900,0,yes,southwest,16884.92400,0,1,3
1,18,male,33.770,1,no,southeast,1725.55230,1,0,2
2,28,male,33.000,3,no,southeast,4449.46200,1,0,2
3,33,male,22.705,0,no,northwest,21984.47061,1,0,1
4,32,male,28.880,0,no,northwest,3866.85520,1,0,1


## 2.2. Filtrado de variables

In [64]:
# Filtering or removing of non desired variables
df_x = df[['age', 'bmi', 'smoker-encoded', 'children', 'sex-encoded', 'region-encoded']]
df_y = df['charges']

# 3. Separación del conjunto de entrenamiento y evaluación

In [65]:
from sklearn import model_selection

In [66]:
from sklearn import preprocessing

## 3.1. Separación de los conjuntos
Es importante notar que, se realiza la separación del conjunto de datos original en **train**, **valid** y **test**, por fuera del framework de Keras para garantizar un adecuado tratamiento de los conjuntos acorde a la metodología empleada. En otras palabras, de esta forma nos aseguramos que cualquier preprocesamiento o normalización sobre validación (valid) y evaluación (test) se realiza a partir de la información obtenida en entrenamiento.

In [67]:
# Split the dataset into train_valid and test
x_train_valid, x_test, y_train_valid, y_test = model_selection.train_test_split(df_x, df_y, test_size=0.2, random_state=1, shuffle=True)

In [68]:
# Split the dataset into train and valid
x_train, x_valid, y_train, y_valid = model_selection.train_test_split(x_train_valid, y_train_valid, test_size=0.3, random_state=1, shuffle=True)

## 3.2. Normalización de variables

In [69]:
# Select the variables where the z-score will be applied
scalable_variables = ['bmi', 'age']

if scalable_variables:
    # Create an instance of the StandardScaler for each variable
    scaler = preprocessing.StandardScaler()

    # Fit the distribution
    scaler.fit(x_train.loc[:, scalable_variables])

    # Transform and normalize all variables
    x_train.loc[:, scalable_variables] = scaler.transform(x_train.loc[:, scalable_variables])
    x_valid.loc[:, scalable_variables] = scaler.transform(x_valid.loc[:, scalable_variables])
    x_test.loc[:, scalable_variables] = scaler.transform(x_test.loc[:, scalable_variables])

## 3.3. Conjuntos de entrenamiento, validación y evaluación
Particularmente, para las redes neuronales de múltiples capas, en este problema vamos a estar utilizando una capa de *embedding* para poder reducir la dimensionalidad necesaria para desacoplar las categorías de la variable *region*. Es decir, normalmente podríamos utilizar un *one hot encoding* para independizar en diferentes variables, pero produce una mayor dimensionalidad y poca interpretabilidad de lo que la red neuronal aprende durante el entrenamiento. En conclusión, debemos separar los grupos de entradas por el formato de entrada de la red neuronal.

In [70]:
x_train = [x_train[['age', 'bmi', 'smoker-encoded', 'children', 'sex-encoded']], x_train['region-encoded']]
x_valid = [x_valid[['age', 'bmi', 'smoker-encoded', 'children', 'sex-encoded']], x_valid['region-encoded']]
x_test = [x_test[['age', 'bmi', 'smoker-encoded', 'children', 'sex-encoded']], x_test['region-encoded']]

# 4. Multilayer Perceptron

In [71]:
import tensorflow.keras as keras

In [72]:
import tensorflow as tf

In [111]:
from src import mlp_helper
importlib.reload(mlp_helper);

In [74]:
model, mae = mlp_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                                  layers_neurons=[6], 
                                  layers_activation=['relu'],
                                  epochs=200,
                                  batch_size=128,
                                  learning_rate=2, 
                                  decay_rate=0.01,
                                  optimizer='adam',
                                  beta_1=0.99,
                                  beta_2=0.999
                                 )

Model logs at tb-logs/mlp/20210523-141636
Model checkpoints at checkpoints/mlp/20210523-141636
Model: "model_26"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_54 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_26 (Embedding)        (None, 1, 2)         8           input_54[0][0]                   
__________________________________________________________________________________________________
input_53 (InputLayer)           [(None, 5)]          0                                            
__________________________________________________________________________________________________
flatten_26 (Flatten)            (None, 2)            0           embedding_26[0][0]            

In [75]:
model, mae = mlp_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                                  layers_neurons=[6, 6], 
                                  layers_activation=['relu', 'relu'], 
                                  epochs=200,
                                  batch_size=128,
                                  learning_rate=2, 
                                  decay_rate=0.01,
                                  optimizer='adam',
                                  beta_1=0.99,
                                  beta_2=0.999
                                 )

Model logs at tb-logs/mlp/20210523-141818
Model checkpoints at checkpoints/mlp/20210523-141818
Model: "model_27"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_56 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_27 (Embedding)        (None, 1, 2)         8           input_56[0][0]                   
__________________________________________________________________________________________________
input_55 (InputLayer)           [(None, 5)]          0                                            
__________________________________________________________________________________________________
flatten_27 (Flatten)            (None, 2)            0           embedding_27[0][0]            

In [76]:
model, mae = mlp_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                                  layers_neurons=[150], 
                                  layers_activation=['relu'], 
                                  epochs=1000,
                                  batch_size=128,
                                  learning_rate=2, 
                                  decay_rate=0.01,
                                  optimizer='adam',
                                  beta_1=0.99,
                                  beta_2=0.999
                                 )

Model logs at tb-logs/mlp/20210523-141951
Model checkpoints at checkpoints/mlp/20210523-141951
Model: "model_28"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_58 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_28 (Embedding)        (None, 1, 2)         8           input_58[0][0]                   
__________________________________________________________________________________________________
input_57 (InputLayer)           [(None, 5)]          0                                            
__________________________________________________________________________________________________
flatten_28 (Flatten)            (None, 2)            0           embedding_28[0][0]            

In [77]:
model, mae = mlp_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                                  layers_neurons=[6, 6, 6], 
                                  layers_activation=['elu', 'elu', 'elu'], 
                                  epochs=500,
                                  batch_size=128,
                                  learning_rate=0.8, 
                                  decay_rate=0.05,
                                  optimizer='adam',
                                  beta_1=0.99,
                                  beta_2=0.999
                                 )

Model logs at tb-logs/mlp/20210523-142131
Model checkpoints at checkpoints/mlp/20210523-142131
Model: "model_29"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_60 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_29 (Embedding)        (None, 1, 2)         8           input_60[0][0]                   
__________________________________________________________________________________________________
input_59 (InputLayer)           [(None, 5)]          0                                            
__________________________________________________________________________________________________
flatten_29 (Flatten)            (None, 2)            0           embedding_29[0][0]            

In [78]:
model, mae = mlp_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                                  layers_neurons=[30, 30, 30], 
                                  layers_activation=['relu', 'relu', 'relu'], 
                                  epochs=200,
                                  batch_size=128,
                                  learning_rate=0.5, 
                                  decay_rate=0.01,
                                  optimizer='adam',
                                  beta_1=0.99,
                                  beta_2=0.999
                                 )

Model logs at tb-logs/mlp/20210523-142247
Model checkpoints at checkpoints/mlp/20210523-142247
Model: "model_30"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_62 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_30 (Embedding)        (None, 1, 2)         8           input_62[0][0]                   
__________________________________________________________________________________________________
input_61 (InputLayer)           [(None, 5)]          0                                            
__________________________________________________________________________________________________
flatten_30 (Flatten)            (None, 2)            0           embedding_30[0][0]            

In [79]:
model, mae = mlp_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                                  layers_neurons=[20, 20, 20, 20], 
                                  layers_activation=['relu', 'relu', 'relu', 'relu'], 
                                  epochs=200,
                                  batch_size=128,
                                  learning_rate=0.3, 
                                  decay_rate=0.01,
                                  optimizer='adam',
                                  beta_1=0.99,
                                  beta_2=0.999
                                 )

Model logs at tb-logs/mlp/20210523-142421
Model checkpoints at checkpoints/mlp/20210523-142421
Model: "model_31"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_64 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_31 (Embedding)        (None, 1, 2)         8           input_64[0][0]                   
__________________________________________________________________________________________________
input_63 (InputLayer)           [(None, 5)]          0                                            
__________________________________________________________________________________________________
flatten_31 (Flatten)            (None, 2)            0           embedding_31[0][0]            

In [80]:
model, mae = mlp_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                                  layers_neurons=[20, 20, 20, 20, 20], 
                                  layers_activation=['relu', 'relu', 'relu', 'relu', 'relu'], 
                                  epochs=200,
                                  batch_size=128,
                                  learning_rate=0.1, 
                                  decay_rate=0.01,
                                  optimizer='adam',
                                  beta_1=0.99,
                                  beta_2=0.999
                                 )

Model logs at tb-logs/mlp/20210523-142559
Model checkpoints at checkpoints/mlp/20210523-142559
Model: "model_32"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_66 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_32 (Embedding)        (None, 1, 2)         8           input_66[0][0]                   
__________________________________________________________________________________________________
input_65 (InputLayer)           [(None, 5)]          0                                            
__________________________________________________________________________________________________
flatten_32 (Flatten)            (None, 2)            0           embedding_32[0][0]            

In [81]:
model, mae = mlp_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                                  layers_neurons=[30, 30, 30, 30, 30], 
                                  layers_activation=['elu', 'elu', 'elu', 'elu', 'elu'], 
                                  epochs=500,
                                  batch_size=256,
                                  learning_rate=0.1, 
                                  decay_rate=0,
                                  optimizer='adam',
                                  beta_1=0.9,
                                  beta_2=0.999
                                 )

Model logs at tb-logs/mlp/20210523-142738
Model checkpoints at checkpoints/mlp/20210523-142738
Model: "model_33"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_68 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_33 (Embedding)        (None, 1, 2)         8           input_68[0][0]                   
__________________________________________________________________________________________________
input_67 (InputLayer)           [(None, 5)]          0                                            
__________________________________________________________________________________________________
flatten_33 (Flatten)            (None, 2)            0           embedding_33[0][0]            

In [82]:
model, mae = mlp_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                                  layers_neurons=[100, 100], 
                                  layers_activation=['elu', 'elu'], 
                                  epochs=1000,
                                  batch_size=256,
                                  learning_rate=0.3, 
                                  decay_rate=0.01,
                                  optimizer='adam',
                                  beta_1=0.99,
                                  beta_2=0.999
                                 )

Model logs at tb-logs/mlp/20210523-142918
Model checkpoints at checkpoints/mlp/20210523-142918
Model: "model_34"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_70 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_34 (Embedding)        (None, 1, 2)         8           input_70[0][0]                   
__________________________________________________________________________________________________
input_69 (InputLayer)           [(None, 5)]          0                                            
__________________________________________________________________________________________________
flatten_34 (Flatten)            (None, 2)            0           embedding_34[0][0]            

In [83]:
model, mae = mlp_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                                  layers_neurons=[50, 50, 50], 
                                  layers_activation=['elu', 'elu', 'elu'], 
                                  epochs=1000,
                                  batch_size=256,
                                  learning_rate=0.2, 
                                  decay_rate=0.01,
                                  optimizer='adam',
                                  beta_1=0.99,
                                  beta_2=0.999
                                 )

Model logs at tb-logs/mlp/20210523-143104
Model checkpoints at checkpoints/mlp/20210523-143104
Model: "model_35"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_72 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_35 (Embedding)        (None, 1, 2)         8           input_72[0][0]                   
__________________________________________________________________________________________________
input_71 (InputLayer)           [(None, 5)]          0                                            
__________________________________________________________________________________________________
flatten_35 (Flatten)            (None, 2)            0           embedding_35[0][0]            

In [84]:
model, mae = mlp_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                                  layers_neurons=[100, 100, 100], 
                                  layers_activation=['elu', 'elu', 'elu'], 
                                  epochs=1000,
                                  batch_size=512,
                                  learning_rate=0.2, 
                                  decay_rate=0.01,
                                  optimizer='adam',
                                  beta_1=0.9,
                                  beta_2=0.999
                                 )

Model logs at tb-logs/mlp/20210523-143339
Model checkpoints at checkpoints/mlp/20210523-143339
Model: "model_36"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_74 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_36 (Embedding)        (None, 1, 2)         8           input_74[0][0]                   
__________________________________________________________________________________________________
input_73 (InputLayer)           [(None, 5)]          0                                            
__________________________________________________________________________________________________
flatten_36 (Flatten)            (None, 2)            0           embedding_36[0][0]            

In [91]:
model, mae = mlp_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                                  layers_neurons=[1500], 
                                  layers_activation=['elu'], 
                                  epochs=1000,
                                  batch_size=64,
                                  learning_rate=0.1, 
                                  decay_rate=0.001,
                                  optimizer='adam',
                                  beta_1=0.8,
                                  beta_2=0.9
                                 )

Model logs at tb-logs/mlp/20210523-144554
Model checkpoints at checkpoints/mlp/20210523-144554
Model: "model_43"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_88 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_43 (Embedding)        (None, 1, 2)         8           input_88[0][0]                   
__________________________________________________________________________________________________
input_87 (InputLayer)           [(None, 5)]          0                                            
__________________________________________________________________________________________________
flatten_43 (Flatten)            (None, 2)            0           embedding_43[0][0]            

In [101]:
model, mae = mlp_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                                  layers_neurons=[1000], 
                                  layers_activation=['tanh'], 
                                  epochs=1000,
                                  batch_size=64,
                                  learning_rate=0.1, 
                                  decay_rate=0.01,
                                  optimizer='adam',
                                  beta_1=0.9,
                                  beta_2=0.99
                                 )

Model logs at tb-logs/mlp/20210523-150126
Model checkpoints at checkpoints/mlp/20210523-150126
Model: "model_53"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_108 (InputLayer)          [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_53 (Embedding)        (None, 1, 2)         8           input_108[0][0]                  
__________________________________________________________________________________________________
input_107 (InputLayer)          [(None, 5)]          0                                            
__________________________________________________________________________________________________
flatten_53 (Flatten)            (None, 2)            0           embedding_53[0][0]            

In [102]:
model, mae = mlp_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                                  layers_neurons=[1000], 
                                  layers_activation=['sigmoid'], 
                                  epochs=1000,
                                  batch_size=64,
                                  learning_rate=0.1, 
                                  decay_rate=0.01,
                                  optimizer='adam',
                                  beta_1=0.9,
                                  beta_2=0.99
                                 )

Model logs at tb-logs/mlp/20210523-150504
Model checkpoints at checkpoints/mlp/20210523-150504
Model: "model_54"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_110 (InputLayer)          [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_54 (Embedding)        (None, 1, 2)         8           input_110[0][0]                  
__________________________________________________________________________________________________
input_109 (InputLayer)          [(None, 5)]          0                                            
__________________________________________________________________________________________________
flatten_54 (Flatten)            (None, 2)            0           embedding_54[0][0]            

In [120]:
model, mae = mlp_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                                  layers_neurons=[1000, 1000, 1000], 
                                  layers_activation=['elu', 'elu', 'elu'], 
                                  use_batch_normalization=True,
                                  epochs=1000,
                                  batch_size=64,
                                  learning_rate=0.1, 
                                  decay_rate=0.01,
                                  optimizer='adam',
                                  beta_1=0.9,
                                  beta_2=0.99
                                 )

Model logs at tb-logs/mlp/20210523-154610
Model checkpoints at checkpoints/mlp/20210523-154610
Model: "model_63"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_134 (InputLayer)          [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_66 (Embedding)        (None, 1, 2)         8           input_134[0][0]                  
__________________________________________________________________________________________________
input_133 (InputLayer)          [(None, 5)]          0                                            
__________________________________________________________________________________________________
flatten_66 (Flatten)            (None, 2)            0           embedding_66[0][0]            

In [121]:
model, mae = mlp_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                                  layers_neurons=[500, 500, 500, 500, 500, 500], 
                                  layers_activation=['elu', 'elu', 'elu', 'elu', 'elu', 'elu'], 
                                  use_batch_normalization=True,
                                  epochs=1000,
                                  batch_size=64,
                                  learning_rate=0.1, 
                                  decay_rate=0.01,
                                  optimizer='adam',
                                  beta_1=0.9,
                                  beta_2=0.99
                                 )

Model logs at tb-logs/mlp/20210523-155106
Model checkpoints at checkpoints/mlp/20210523-155106
Model: "model_64"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_136 (InputLayer)          [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_67 (Embedding)        (None, 1, 2)         8           input_136[0][0]                  
__________________________________________________________________________________________________
input_135 (InputLayer)          [(None, 5)]          0                                            
__________________________________________________________________________________________________
flatten_67 (Flatten)            (None, 2)            0           embedding_67[0][0]            

In [123]:
model, mae = mlp_helper.run_model(x_train, y_train, x_valid, y_valid, x_test, y_test,
                                  layers_neurons=[200, 100, 50], 
                                  layers_activation=['elu', 'elu', 'elu'], 
                                  use_batch_normalization=True,
                                  epochs=10000,
                                  batch_size=128,
                                  learning_rate=0.3, 
                                  decay_rate=0.001,
                                  optimizer='adam',
                                  beta_1=0.85,
                                  beta_2=0.99,
                                  patience=200,
                                  min_delta=5
                                 )

Model logs at tb-logs/mlp/20210523-162945
Model checkpoints at checkpoints/mlp/20210523-162945
Model: "model_66"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_140 (InputLayer)          [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_69 (Embedding)        (None, 1, 2)         8           input_140[0][0]                  
__________________________________________________________________________________________________
input_139 (InputLayer)          [(None, 5)]          0                                            
__________________________________________________________________________________________________
flatten_69 (Flatten)            (None, 2)            0           embedding_69[0][0]            